# LSTM Forecasting Model

O'Reilly Machine Learning and Security
* https://github.com/oreilly-mlsec/mlsec.net

Chapter 3 Resources
* https://github.com/oreilly-mlsec/book-resources/blob/master/chapter3/lstm-anomaly-detection.ipynb

In [2]:
import numpy as np

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Activation, Dropout

from collections import deque

from utils import *

Using TensorFlow backend.


## Data Loading and Standardization

In [ ]:
print("loading data...")
reader = read_tcpdump_file('data/week1_friday.tcpdump')
packets = np.array([f for f in featurize_packets(reader)])

print("standardizing data... (impossible given protocol is invariable)")
means = np.apply_along_axis(np.mean, 0, packets)
stds = np.apply_along_axis(np.std, 0, packets)
packets -= means
packets /= stds

print("done.")

# TODO: cull non-continuous variables OR manually define loss

## Hyper-parameters

In [41]:
epochs = 10
batch_size = 50
sequence_length = 4
features = 14
mean_window = 40
loss_tolerance = 2
train_test_split = 8

## Class Definition

In [37]:
class ForecastModel(object):
    
    def __init__(self):
        self.model = self.generate_model()
    
    def generate_model(self):
        
        model = Sequential()
        
        model.add(LSTM(input_shape=(sequence_length - 1, features), 
                       units=32, 
                       return_sequences=True))
        model.add(Dropout(0.2))
        
        model.add(LSTM(units=128,
                       return_sequences=True))
        model.add(Dropout(0.2))

        model.add(LSTM(units=100,
                       return_sequences=False))
        model.add(Dropout(0.2))
        
        model.add(Dense(units=features))
        model.add(Activation('linear'))
        
        model.compile(loss='mean_squared_error', optimizer='rmsprop')
        
        return model
    
    def prepare_data(self, data, training=True):
        
        if training:

            print('creating train n-grams...')

            train_grams = []
            for i in range(0, len(data) - sequence_length):
                train_grams.append(data[i: i + sequence_length])
            train_grams = np.array(train_grams)

            print('train data shape : ', train_grams.shape)

            self.x_train = train_grams[:, :-1]
            self.y_train = train_grams[:, -1]

        else:

            print('creating test n-grams...')

            test_grams = []
            for i in range(0, len(data) - sequence_length):
                test_grams.append(data[i: i + sequence_length])
            test_grams = np.array(test_grams)

            print('test data shape : ', test_grams.shape)  

            self.x_test = test_grams[:, :-1]
            self.y_test = test_grams[:, -1]        
    
    def train(self, data):
        
        self.prepare_data(data)
        self.train_history = self.model.fit(self.x_train, self.y_train,
                                            batch_size=batch_size, epochs=epochs)
        
    def test(self, data):
        
        assert self.train_history is not None 
        
        self.prepare_data(data, training=False)
        
        losses_in_window = deque()
        moving_mean = 0
        rolling_std = 0
        
        for i in range(self.x_test.shape[0]):
            
            test_loss = self.model.evaluate(x=np.expand_dims(self.x_test[i], 0), 
                                            y=np.expand_dims(self.y_test[i], 0), batch_size=1)
            
            if i < mean_window:
                moving_mean += test_loss / mean_window
                losses_in_window.append(test_loss)
                print("build mean")
            else:
                moving_mean += (test_loss - losses_in_window[0]) / mean_window
                losses_in_window.popleft()
                losses_in_window.append(test_loss)
                
                # not efficient
                rolling_std = np.std(losses_in_window)
                
                if np.abs(test_loss - moving_mean) < rolling_std * loss_tolerance:
                    print("all clear")
                else:
                    print("\nMALICIOUS\n")

In [38]:
model = ForecastModel()

split = (len(packets) * train_test_split) // 10

model.train(packets[:split])
model.test(packets[split:])

creating train n-grams...
train data shape :  (96, 4, 14)
Epoch 1/10
96/96 [==============================] - 3s 29ms/step - loss: 659571.1680
Epoch 2/10
96/96 [==============================] - 0s 353us/step - loss: 659311.6862
Epoch 3/10
96/96 [==============================] - 0s 434us/step - loss: 658754.6549
Epoch 4/10
96/96 [==============================] - 0s 365us/step - loss: 657576.6107
Epoch 5/10
96/96 [==============================] - 0s 534us/step - loss: 655627.9609
Epoch 6/10
96/96 [==============================] - 0s 492us/step - loss: 653331.4115
Epoch 7/10
96/96 [==============================] - 0s 469us/step - loss: 652108.1862
Epoch 8/10
96/96 [==============================] - 0s 416us/step - loss: 651500.1146
Epoch 9/10
96/96 [==============================] - 0s 527us/step - loss: 651040.4492
Epoch 10/10
96/96 [==============================] - 0s 441us/step - loss: 650634.9310
